# Imports

Для того, чтобы ноутбук получился структурированным давайте вынесем все imports вверх. Это хорошая DS практика, которая поможет написать читаемый и легко интерпретируемый код. Поверьте, ваши коллеги будут вам благодарны.

In [ ]:
# отключим warnings, чтобы они нам не мешали
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# подгрузим стандартные библиотеки
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# подгрузим много разных ML инструментов
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.utils import compute_class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Downloading the dataset

В этом вебинаре мы будем использовать Kaggle [датасет](https://www.kaggle.com/elikplim/car-evaluation-data-set), посвященный оценке безопасности того или иного автомобиля.

Давайте подгрузим данные с Kaggle прямо в Colab, используя API Kaggle.

In [ ]:
os.environ['KAGGLE_USERNAME'] = "soncaajp"
os.environ['KAGGLE_KEY'] = "d3783c48119342658b6e9b4a4a190ce6"

Подгружаем датасет.

In [ ]:
! kaggle datasets download -d elikplim/car-evaluation-data-set

  0% 0.00/4.66k [00:00<?, ?B/s]
100% 4.66k/4.66k [00:00<00:00, 8.86MB/s]


Распаковываем подгруженные данные.

In [ ]:
! unzip /content/car-evaluation-data-set.zip

Archive:  /content/car-evaluation-data-set.zip
  inflating: car_evaluation.csv      


# Exploratory data analysis (EDA)

Подгрузим распакованные данные.

In [ ]:
data = pd.read_csv('car_evaluation.csv', header = None)
data.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


Выглядит очень странно. Непонятно за что отвечает тот или иной столбец. Давайте переименуем столбцы так, как указано на страничке датасета на Kaggle.

In [ ]:
col_names = ['Buying_price', 'Maintenance_cost', 'Number_of_doors', 'Number_of_persons', 'Luggage_boot', 'Safety', 'Class']
data.columns = col_names
data.head()

,Buying_price,Maintenance_cost,Number_of_doors,Number_of_persons,Luggage_boot,Safety,Class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


Давайте проведем EDA.

На что стоит обратить внимание:

1.   Размер датасета
2.   Типы переменных в датасете
3.   Наличие пропусков
4.   Распределение признаков и их "смысл"
5.   Тип задачи Машинного обучения

In [ ]:
# какого размера наш датасет?

In [ ]:
for column in data.columns:
    print(data[column].value_counts())  # какой вывод можно сделать по выводу этой ячейки?
    print()                             # что можно сказать про признаки? а про задачу машинного обучения? а про баланс классов?

low      432
high     432
vhigh    432
med      432
Name: Buying_price, dtype: int64

low      432
high     432
vhigh    432
med      432
Name: Maintenance_cost, dtype: int64

2        432
5more    432
3        432
4        432
Name: Number_of_doors, dtype: int64

2       576
more    576
4       576
Name: Number_of_persons, dtype: int64

big      576
small    576
med      576
Name: Luggage_boot, dtype: int64

low     576
high    576
med     576
Name: Safety, dtype: int64

unacc    1210
acc       384
good       69
vgood      65
Name: Class, dtype: int64



In [ ]:
# сколько пропусков в нашем датасете

In [ ]:
# с какой задачей машинного обучения мы столкнулись?



Давайте разобьем нашу выборку на train и test в соотношении 70 на 30, используя стратификацию.

In [ ]:
X = data.drop(['Class'], axis = 1)
y = data['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

Посмотрим на баланс классов в тренировочной и валидационной выборке.

In [ ]:
print(y_train.shape)
print(y_train.value_counts())

(1209,)
unacc    847
acc      269
good      48
vgood     45
Name: Class, dtype: int64


In [ ]:
print(y_test.shape)
print(y_test.value_counts())

(519,)
unacc    363
acc      115
good      21
vgood     20
Name: Class, dtype: int64


Признаки будем кодировать с помощью Ordinal Encoder из sklearn.preprocessing. Мы могли бы сделать это вручную, как в случае с кодированием признака ecology в ML-2, но это заняло бы какое-то время.

In [ ]:
X_train.head()

,Buying_price,Maintenance_cost,Number_of_doors,Number_of_persons,Luggage_boot,Safety
101,vhigh,vhigh,5more,more,small,high
844,high,low,5more,2,big,med
1361,low,vhigh,4,4,small,high
1584,low,med,4,more,small,low
566,high,high,2,more,big,high


In [ ]:
encoder = OrdinalEncoder()
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [ ]:
encoder.categories_

[array(['high', 'low', 'med', 'vhigh'], dtype=object),
 array(['high', 'low', 'med', 'vhigh'], dtype=object),
 array(['2', '3', '4', '5more'], dtype=object),
 array(['2', '4', 'more'], dtype=object),
 array(['big', 'med', 'small'], dtype=object),
 array(['high', 'low', 'med'], dtype=object)]

In [ ]:
X_train = pd.DataFrame(data = X_train, columns = ['Buying_price', 'Maintenance_cost', 'Number_of_doors', 'Number_of_persons', 'Luggage_boot', 'Safety'])
X_test = pd.DataFrame(data = X_test, columns = ['Buying_price', 'Maintenance_cost', 'Number_of_doors', 'Number_of_persons', 'Luggage_boot', 'Safety'])

In [ ]:
X_train.head()

,Buying_price,Maintenance_cost,Number_of_doors,Number_of_persons,Luggage_boot,Safety
0,3.0,3.0,3.0,2.0,2.0,0.0
1,0.0,1.0,3.0,0.0,0.0,2.0
2,1.0,3.0,2.0,1.0,2.0,0.0
3,1.0,2.0,2.0,2.0,2.0,1.0
4,0.0,0.0,0.0,2.0,0.0,0.0


In [ ]:
X_train['Buying_price'].value_counts()

0.0    325
3.0    307
1.0    293
2.0    284
Name: Buying_price, dtype: int64

Таким образом, мы успешно закодировали все ординальные признаки в нашем датасете. Целевую переменную можно оставить как есть. Почему?


Итак, наши данные готовы для обучения. Мы работаем с задачей многоклассовой Классификации, причем имеем достаточно сильный перекос в целевой переменной в сторону класса **unacc**. Работать будем по следующему алгоритму:



1.   Наивная модель
2.   Logistic Regression
3.   Decision Tree
4.   Random Forest
5.   СatBoost

Попробуем выбить максимальный возможный **accuracy**.



# Наивная модель

Давайте всегда предсказывать только класс-большинство, а именно **unacc**. Accuracy, полученный таким способом, будет нашим **baseline** - значением метрики, которую нужно побить, используя более сложные подходы.

In [ ]:
predict = # cформируйте массив, содержащий строку 'unacc' y_test.shape[0] раз

In [ ]:
len(predict) # проверьте, что размерность сформированного массива совпадает с нужной (должна быть 519)

519

In [ ]:
print('Accuracy наивной модели: ') # посчитайте accuracy между сформированным массивом и y_test

Мы получили baseline. Понятно, что ценности в таком решении нет и нужно работать дальше.

# Logistic Regression

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
lr = # инициализируйте модель LogisticRegression
lr.# обучите модель, которую вы инициализировали выше на X_train, y_train
predict = # получите предсказания модели на X_test
print('Accuracy out-of-the-box Logistic Regression: ') # посчитайте accuracy модели на predict (то есть X_test)

Несмотря на то, что **accuracy** получилась меньше, чем при наивном решении, стоит дополнительно оценить модель с помощью **confusion_matrix**.

In [ ]:
cm = confusion_matrix(y_test, predict, labels = list(y.unique()))
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = list(y.unique()))
disp.plot()

# Decision Tree

Давайте отойдем от линейных моделей и попробуем применить к нашей задаче алгоритм **Решающего дерева**. К сожалению, даже если мы получим хорошую метрику **accuracy**, мы не сможем использовать данную модель, поскольку, как вы знаете, **Decision Tree** склонен сильно переобучаться. Однако, такой проверкой мы сможем подтвердить, что Решающие деревья в целом применимы для решения нашего кейса.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

In [ ]:
# используйте OrdinalEncoder, чтобы преобразовать признаки (код перекодировки смотрите выше в части EDA)

In [ ]:
dtc = # инициализируйте модель DecisionTreeClassifier с random_state 42
dtc.# обучите модель, которую вы инициализировали выше на X_train, y_train
predict = # получите предсказания модели на X_test
print('Accuracy DecisionTreeClassifier на train: ') # посчитайте accuracy модели на X_train
print('Accuracy DecisionTreeClassifier на test: ') # посчитайте accuracy модели на predict (то есть X_test)

In [ ]:
cm = confusion_matrix(y_test, predict, labels = list(y.unique()))
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = list(y.unique()))
disp.plot()

Почему мы вообще могли пропустить шаг с Decision Tree?

# Random Forest

Применим RandomForestClassifier.

In [ ]:
rfc = # инициализируйте модель RandomForestClassifier с random_state 42
rfc.# обучите модель, которую вы инициализировали выше на X_train, y_train
predict = # получите предсказания модели на X_test
print('Accuracy RandomForestClassifier на train: ')) # посчитайте accuracy модели на X_train
print('Accuracy RandomForestClassifier на test: ') # посчитайте accuracy модели на predict (то есть X_test)

In [ ]:
cm = confusion_matrix(y_test, predict, labels = list(y.unique()))
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = list(y.unique()))
disp.plot()